## Documentation Data Reader

This Jupyter notebook is meant to serve as an introduction to reading Github `.md` documentation and analyzing it..

### Step 1: Reading and Storing the Documentation Data

In this section, we'll read the markdown `.md` file data, collect it, and store it for processing. We can do this by reading through all of the `.md` files in a directory and reading them into plain text format, then storing it.

In [1]:
import markdown
from bs4 import BeautifulSoup

def read_md_file(filepath: str) -> str:
    """
    Reads a markdown file and processes it into a string of plain text.

    Parameters:
        filepath (str) : the filepath of the markdown file to read
    
    Returns:
        text (str) : the plain text from the inputted markdown file
    """
    with open(filepath, 'r') as f:
        content = f.read()
        html = markdown.markdown(content)
        text = ''.join(BeautifulSoup(html).findAll(text=True))

    return text


In [2]:
text = read_md_file("../intro/proj-overview.md")

/var/folders/t9/lp_w7fr92mj40y7_28ynvst40000gn/T/ipykernel_33684/3585446199.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = ''.join(BeautifulSoup(html).findAll(text=True))


In [7]:
import os

def collect_doc_data(directory: str) -> list[str]:
    """
    Scans through a directory and collects the documentation data from all
    '.md' files into a list.

    Parameters:
        directory (str) : directory to scan through
    
    Returns
        docs_data (list[str]) : documentation data from `.md` files
    """
    doc_data = []
    for dirpath, _, filenames in os.walk(directory):
        for file in filenames:
            if file.endswith('.md'):
                file_path = os.path.join(dirpath, file)
                text = read_md_file(file_path)
                doc_data.append(text)
    
    return doc_data


In [8]:
doc_data = collect_doc_data("../docs/docs")

/var/folders/t9/lp_w7fr92mj40y7_28ynvst40000gn/T/ipykernel_33684/3585446199.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = ''.join(BeautifulSoup(html).findAll(text=True))


### Step 2: Cleaning the Documentation Data

In this section, we'll take our collected and stored documentation data from Step 1 and clean it up so we can use it. This could include removing HTML tags, removing punctuation and special characters, removing extra whitespaces from the text, making all of our text lowercase for semantic searching, and catching any mispellings in the documentation.

In [ ]:
def clean_doc_data(doc_data: list[str]) -> list[str]:
    """
    Clean the documentation data by removing HTML tags, removing punctuation
    and special characters, removing extra whitespaces, making everything
    lowercase, and catching mispelled words.

    Parameters:
        doc_data (list[str]) : the collected and read `.md` data
    
    Returns:
        cleaned_doc_data (list[str]) : the cleaned documentation data
    """
    raise NotImplementedError


### Step 3: Pre-processing the Documentation Data

In this section, we'll take our cleaned documentation data from Step 2 and pre-process it by tokenization, stemming lemmatization, and stop-word removal.

In [10]:
def preprocess_doc_data(cleaned_doc_data: list[str]) -> list[str]:
    """
    Preprocess the documentation data by tokenization, stemming
    lemmatization, and stop-word removal.

    Parameters:
        cleaned_doc_data (list[str]) : the cleaned documentation data
    
    Returns:
        proc_doc_data (list[str]) : preprocessed documentation data
    """
    raise NotImplementedError